In [4]:
import sys
sys.path.insert(0, '../../../network')

In [5]:
import logging
import argparse
import numpy as np

from network import Population, RateNetwork
from transfer_functions import ErrorFunction
from connectivity import SparseConnectivity, LinearSynapse
from sequences import GaussianSequence

logging.basicConfig(level=logging.INFO)

In [7]:
T = 0.35

phi = ErrorFunction(mu=0.22, sigma=0.1).phi
exc = Population(N=40000, tau=1e-2, phi=phi)

S, P = 1, 16
sequences = [GaussianSequence(P,exc.size,seed=i) for i in range(S)]
patterns = np.stack([s.inputs for s in sequences])

conn_EE = SparseConnectivity(source=exc, target=exc, p=0.005)
synapse = LinearSynapse(conn_EE.K, A=1)
conn_EE.store_sequences(patterns, synapse.h_EE)

net = RateNetwork(exc, c_EE=conn_EE, formulation=1)

logging.info("Simulating normal retrieval")
r0 = exc.phi(patterns[0,0,:])
net.simulate(T, r0=r0)
M = np.mean(net.exc.state**2, axis=0)
overlaps = sequences[0].overlaps(net, exc)
correlations = sequences[0].overlaps(net, exc, correlation=True)
phi_patterns = phi(patterns)
state = np.copy(net.exc.state)

net.clear_state()
logging.info("Simulating perturbed retrieval")
sigma_pert = 0.75
r0 = exc.phi(patterns[0,0,:] + sigma_pert*np.random.randn(exc.size))
net.simulate(T, r0=r0)
M_pert = np.mean(net.exc.state**2, axis=0)
overlaps_pert = sequences[0].overlaps(net, exc)
correlations_pert = sequences[0].overlaps(net, exc, correlation=True)
state_pert = net.exc.state

np.save(
    open("data/data.npy", "wb"),
    [M, overlaps, correlations, state.astype(np.float32),
     M_pert, overlaps_pert, correlations_pert, state_pert.astype(np.float32),
     phi_patterns.astype(np.float32)])

INFO:connectivity:Building connections from exc to exc
INFO:connectivity:Storing sequences
100%|██████████| 1/1 [00:01<00:00,  1.79s/it]414.99it/s]
INFO:connectivity:Applying synaptic transfer function
INFO:connectivity:Building sparse matrix
INFO:root:Simulating normal retrieval
INFO:network:Integrating network dynamics
[========================================================================] 100%
INFO:sequences:Computing overlaps
100%|██████████| 16/16 [00:00<00:00, 162.84it/s]
INFO:sequences:Computing correlations
100%|██████████| 16/16 [00:07<00:00,  2.09it/s]
INFO:root:Simulating perturbed retrieval
INFO:network:Integrating network dynamics
[========================================================================] 100%
INFO:sequences:Computing overlaps
100%|██████████| 16/16 [00:00<00:00, 142.96it/s]
INFO:sequences:Computing correlations
100%|██████████| 16/16 [00:07<00:00,  2.13it/s]
